You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
import sys

print("Hello world")

print(f"Python version: {sys.version}")



Hello world
Python version: 3.11.9 (main, Apr 19 2024, 16:48:06) [GCC 11.2.0]


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [12]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [13]:
result = crew.kickoff(inputs={"topic": "CrewAI pricing models"}) 

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on CrewAI pricing models.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
Content Plan

Title: Navigating CrewAI Pricing Models: Everything You Need to Know

Introduction:
- Briefly introduce CrewAI and its significance in the AI industry.
- Mention the focus of the article on exploring the various pricing models offered by CrewAI.

Key Points:
I. Trending Pricing Models in CrewAI
   A. Pay-As-You-Go Model
      1. Description and benefits
      2. Pricing structure and flexibility
   B. Subscription Model
      1. Overview and advantages
      2. Different tiers and features
   C. Enterprise/Custom Pricing
      1. Tailored solutions for large businesses
      2.

- Display the results of your execution as markdown in the notebook.

In [27]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Mystery of CrewAI Pricing Models

## Introduction

In the rapidly evolving landscape of artificial intelligence (AI) technology, understanding pricing models is crucial for businesses looking to leverage AI solutions effectively. One key player in the AI market offering competitive pricing models is CrewAI. By delving into the intricacies of CrewAI's pricing structures, businesses can make informed decisions on implementing AI solutions that align with their budget and goals.

## Latest Trends and Key Players

The tech industry is witnessing a surge in innovative AI pricing models, with key players like CrewAI at the forefront of offering cutting-edge solutions. As businesses increasingly rely on AI tools for various applications, staying abreast of the latest trends in pricing strategies is essential. CrewAI's competitive pricing models cater to the diverse needs of businesses, providing affordable AI solutions without compromising on quality.

## Noteworthy News on CrewAI Pricing Models

Recent updates on CrewAI's pricing structure have highlighted the company's commitment to delivering value-driven AI solutions. Customer reviews and feedback on CrewAI's pricing models have been overwhelmingly positive, underscoring the company's reputation for transparency and customer satisfaction. With new features and offerings in the pipeline, businesses can expect continued innovation from CrewAI in the realm of AI pricing models.

## Target Audience Analysis

For small to medium-sized businesses, marketing professionals, and entrepreneurs seeking cost-effective AI tools, understanding AI pricing models is paramount. By analyzing CrewAI's pricing options, businesses can tailor their AI strategies to meet their unique needs and budget constraints. Entrepreneurs looking to integrate AI solutions into their operations can benefit from insights into AI pricing trends and strategies for maximizing ROI.

## Conclusion

In conclusion, navigating the complex landscape of AI pricing models requires a comprehensive understanding of the latest trends, key players, and industry news. By exploring CrewAI's pricing models and offerings, businesses can unlock the full potential of AI technology while staying within budget constraints. As the AI market continues to evolve, staying informed on pricing strategies will be essential for businesses looking to stay competitive and drive innovation in their respective industries.

## Call to Action

Ready to explore CrewAI's pricing options and revolutionize your business with AI solutions? Visit CrewAI's website for more information and request a demo today. Join us for a webinar or workshop on AI pricing strategies to gain valuable insights from industry experts. For further inquiries or consultations on AI pricing models, fill out our contact form and let us help you navigate the future of AI technology.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [26]:
# topic = "YOUR TOPIC HERE"
# result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).